In [1]:
# Función para generar la ruta de S3 a partir de la fecha y el productodef build_goes17_s3_path(product, satellite, date):     year = date.strftime("%Y")     day_of_year = date.strftime("%j")     hour = date.strftime("%H") file_name = f"OR_{product}-M6C02_G{satellite}_s{year}{day_of_year}{hour}0000.nc" s3_path = f"s3://noaa-goes17/{product}/{year}/{day_of_year}/{hour}/{file_name}"return s3_path

In [2]:
import os
import GOES
import xarray as xr
import datetime as dt
import shutil
import os

def generar_intervalos(fecha_inicio, fecha_fin):
    tiempo_inicio = dt.datetime.strptime(fecha_inicio, '%Y%m%d-%H%M%S')
    tiempo_fin = dt.datetime.strptime(fecha_fin, '%Y%m%d-%H%M%S')
    delta = dt.timedelta(minutes=10)
    intervalos = []
    while tiempo_inicio <= tiempo_fin:
        intervalos.append(tiempo_inicio.strftime('%Y%m%d%H%M'))
        tiempo_inicio+=delta
    print('intervalos:',intervalos)
    print('cantidad intervalos:', len(intervalos))
    return intervalos

In [3]:

def buscar_imagenes_faltantes_explicitamente(intervalos_esperados, timestamps_archivos):
    imagenes_faltantez2 = []
    
    if not isinstance(intervalos_esperados, list) or not isinstance(timestamps_archivos, list):
        print("Error: intervalos_esperados o timestamps_archivos no son listas.")
        return []

    for intervalo in intervalos_esperados:
        encontrado = False
        for timestamp in timestamps_archivos:
            if intervalo == timestamp:
                encontrado = True
                print(f"Comparación: {intervalo} == {timestamp} -> Encontrado")
                break
        if not encontrado:
            print(f"Comparación: {intervalo} no está en timestamps_archivos -> Faltante")
            imagenes_faltantez2.append(intervalo)
    
    print("Imágenes faltantes:", imagenes_faltantez2)
    print("Cantidad imágenes faltantes:", len(imagenes_faltantez2))

    return imagenes_faltantez2



def buscar_imagenes_faltantes(carpeta, fecha_hora_inicio, fecha_hora_fin, toggle=False):
    intervalos_esperados = generar_intervalos(fecha_hora_inicio, fecha_hora_fin)
    

    if not os.path.exists(carpeta):
        print(f"Error: la carpeta {carpeta} no existe.")
        return [], [], []
    
    archivos = os.listdir(carpeta)
    print("Archivos en la carpeta:", archivos)
    
    timestamps_archivos = []
    for archivo in archivos:
        if (archivo.startswith('OR_ABI-L2-ACMF-M6_G16_s') or archivo.startswith('OR_ABI-L2-ACMF-M4_G16_s')) and archivo.endswith('.png'):
            # Extraer la parte del nombre del archivo que contiene el timestamp
            timestamp = archivo.split('_')[3][1:13]  # Extraer 202305020330 del nombre
            if timestamp in intervalos_esperados:
                #print('fecha guardad en timestamps',timestamp)
                timestamps_archivos.append(timestamp) # listado de fechas que si tienen imagenes
            

    print("extraidos:",timestamps_archivos)
    print("cantidad extraidos:",len(timestamps_archivos))

    imagenes_faltantez = buscar_imagenes_faltantes_explicitamente(intervalos_esperados, timestamps_archivos)

    imagenes_faltantes = [intervalo[:8] + '-' + intervalo[8:] + '00' for intervalo in imagenes_faltantez]

    print("Imágenes faltantes:", imagenes_faltantez)
    print("cantidad imagenes faltantes", len(imagenes_faltantez))

    return imagenes_faltantes, intervalos_esperados, timestamps_archivos,imagenes_faltantez

    
def download_crop_save_goes(satelite, producto, fecha_hora_inicio, fecha_hora_fin, canales, formato_fecha, carpeta_almacenamiento_images, carpeta_descarga,x_slice, y_slice):
        
    intervalos, imagenes_faltantez, estampas, noestan = buscar_imagenes_faltantes(carpeta_almacenamiento_images, fecha_hora_inicio, fecha_hora_fin)
    fecha_hora_fin_dt = dt.datetime.strptime(fecha_hora_fin, '%Y%m%d-%H%M%S')-dt.timedelta(minutes=10)

    carpeta_tratamiento = 'C:/Users/guavitaj/Documents/tesis/algoritmos/modelo_xray/prueba2/'

    for intervalo in intervalos:
        print(f"Descargando datos para el intervalo {intervalo}...")
        
        tiempo_ini = dt.datetime.strptime(intervalo, '%Y%m%d-%H%M%S')
        tiempo_fin = tiempo_ini + dt.timedelta(minutes=10)
        
        print('tiempo ini', tiempo_ini)

        if tiempo_ini >= fecha_hora_fin_dt:
            print(f"Alcanzado el último intervalo: {intervalo}, terminando.")
            break

        m_version = "M6"
        archivo_destino = os.path.join(carpeta_descarga, f'OR_ABI-L2-ACMF-{m_version}_G16_s{intervalo.replace("-", "")}.nc')

        if os.path.exists(archivo_destino):
            print(f"El archivo para el intervalo {intervalo} ya existe. Saltando descarga.")
            continue

        m_version = "M4"
        archivo_destino_m4 = os.path.join(carpeta_descarga, f'OR_ABI-L2-ACMF-{m_version}_G16_s{intervalo.replace("-", "")}.nc')
        
        if os.path.exists(archivo_destino_m4):
            print(f"El archivo para el intervalo {intervalo} ya existe. Saltando descarga.")
            continue
             
        print(f"Descargando intervalo {tiempo_ini} - {tiempo_fin}...")
             
        DateTimeFin = tiempo_fin.strftime('%Y%m%d-%H%M%S')
        
        GOES.download(satelite, producto,
                      DateTimeIni=intervalo, DateTimeFin=DateTimeFin,
                      channel=canales, rename_fmt=formato_fecha, path_out=carpeta_tratamiento)
               
        archivos_nc = [os.path.join(carpeta_tratamiento, f) for f in os.listdir(carpeta_tratamiento) if f.endswith('.nc')]
        print('intervalo',tiempo_ini,'-',tiempo_fin)
        print('Archivos descargados:', archivos_nc)

        for archivo in archivos_nc:
            print(f"Procesando {archivo}...")

            if "M4" in archivo:
                m_version = "M4"
            elif "M6" in archivo:
                m_version = "M6"

            with xr.open_dataset(archivo) as ds:
                
                ds_recortado = ds.sel(x=slice(x_slice[0], x_slice[1]), y=slice(y_slice[0], y_slice[1]))

                temp_output_path = archivo.replace('.nc', '_temp.nc')
                ds_recortado.to_netcdf(temp_output_path)
                print(f"Archivo recortado guardado temporalmente: {temp_output_path}")

            try:
                os.remove(archivo)  
                os.rename(temp_output_path, archivo)
                
                nuevo_nombre = archivo.replace("-", "")
                nuevo_nombre = nuevo_nombre.replace("OR_ABIL2ACMFM6", f"OR_ABI-L2-ACMF-{m_version}").replace("OR_ABIL2ACMFM4", f"OR_ABI-L2-ACMF-{m_version}")

                print(f"Renombrando archivo: {archivo} -> {nuevo_nombre}")
                
                nuevo_path = os.path.join(carpeta_descarga, os.path.basename(nuevo_nombre))             
                shutil.move(archivo, nuevo_path)
                print(f"Archivo movido a: {nuevo_path}")

                
            except PermissionError as e:
                print(f"Error de permiso al intentar eliminar o renombrar: {e}")





In [4]:
carpeta_almacenamiento_images = 'C:/Users/guavitaj/Documents/tesis/algoritmos/modelo_xray/archivos_cortados2/mayo/septiembre'
carpeta_descarga = 'C:/Users/guavitaj/Documents/tesis/algoritmos/modelo_xray/prueba2/'
fecha_hora_inicio = '20230801-000000'
fecha_hora_fin = '20230820-000000'

satelite = 'goes16'                       # Satélite de la serie GOES-R del cual se requiere información
producto = 'ABI-L2-ACMF'                  # Producto del satélite del cual se requiere información
canales = ['']                            # banda si se necesita
formato_fecha = '%Y%m%d-%H%M%S'            # Formato en el cual se almacenará la fecha de los archivos
x_slice = (-0.01, 0.03)
y_slice = (0.05, 0.01)

imagenes_faltantes, int_esperados, estampas2, no_estan = buscar_imagenes_faltantes(carpeta_almacenamiento_images, fecha_hora_inicio, fecha_hora_fin)


if imagenes_faltantes:
    a=1
    #download_crop_save_goes(satelite, producto, fecha_hora_inicio, fecha_hora_fin, canales, formato_fecha, carpeta_almacenamiento_images, carpeta_descarga, x_slice, y_slice)

else:
    print("No faltan imágenes.")

intervalos: ['202308010000', '202308010010', '202308010020', '202308010030', '202308010040', '202308010050', '202308010100', '202308010110', '202308010120', '202308010130', '202308010140', '202308010150', '202308010200', '202308010210', '202308010220', '202308010230', '202308010240', '202308010250', '202308010300', '202308010310', '202308010320', '202308010330', '202308010340', '202308010350', '202308010400', '202308010410', '202308010420', '202308010430', '202308010440', '202308010450', '202308010500', '202308010510', '202308010520', '202308010530', '202308010540', '202308010550', '202308010600', '202308010610', '202308010620', '202308010630', '202308010640', '202308010650', '202308010700', '202308010710', '202308010720', '202308010730', '202308010740', '202308010750', '202308010800', '202308010810', '202308010820', '202308010830', '202308010840', '202308010850', '202308010900', '202308010910', '202308010920', '202308010930', '202308010940', '202308010950', '202308011000', '2023080110

In [7]:
print("no estan:",no_estan)

no estan: ['202308041910', '202308041920', '202308041930', '202308192350', '202308200000']


In [6]:
a=54565regdgfcg

SyntaxError: invalid decimal literal (2606372269.py, line 1)

In [8]:
import datetime
import os
import shutil
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import xarray as xr

# Lista de fechas y horas exactas que no están disponibles
not_in_list = no_estan

def download_and_process_ACM(yyyymmddhhmn, path_local, path_dest, product_name):
    os.makedirs(path_local, exist_ok=True)
    os.makedirs(path_dest, exist_ok=True)

    year = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
    day_of_year = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%j')
    hour = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
    min = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

    # Información del repositorio de AMAZON
    bucket_name = 'noaa-goes16'

    # Inicializa el cliente S3
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

    # Estructura del archivo en el servidor
    prefix = f'{product_name}/{year}/{day_of_year}/{hour}/OR_{product_name}-M6_G16_s{year}{day_of_year}{hour}{min}'

    # Buscar el archivo en el servidor
    s3_result = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter="/")

    # Verificar si hay archivos disponibles
    if 'Contents' not in s3_result:
        # No hay archivos disponibles
        print(f'No files found for the date: {yyyymmddhhmn}')
        return -1
    else:
        # Existen archivos disponibles
        for obj in s3_result['Contents']:
            key = obj['Key']
            # Extraer el nombre del archivo original
            original_file_name = key.split('/')[-1]
            local_file_path = f'{path_local}/{original_file_name}'

            # Descargar el archivo original si no existe
            if os.path.exists(local_file_path):
                print(f'File already exists in {path_local}: {local_file_path}')
            else:
                print(f'Downloading file to {path_local}: {local_file_path}')
                s3_client.download_file(bucket_name, key, local_file_path)

            # Abrimos el archivo y recortamos
            try:
                with xr.open_dataset(local_file_path) as ds:
                    # Recorte basado en las coordenadas especificadas
                    ds_sel = ds.sel(x=slice(-0.01, 0.03), y=slice(0.05, 0.01))  # Ajusta las coordenadas según tu necesidad
                    
                    # Guardar el archivo recortado con el nombre original
                    cropped_file_path = f'{path_dest}/{original_file_name}'
                    ds_sel.to_netcdf(cropped_file_path)
                    print(f'Cropped and saved file as {cropped_file_path}')

                    # Renombrar y guardar con nuevo formato
                    start_time_str = original_file_name.split('_')[3][1:]  # Obtiene la parte sYYYYJJJHHMMSS
                    year_from_file = start_time_str[:4]
                    day_of_year_from_file = start_time_str[4:7]
                    hour_from_file = start_time_str[7:9]
                    minute_from_file = start_time_str[9:11]
                    second_from_file = start_time_str[11:13]

                    # Convertir la fecha del archivo en formato 'YYYYMMDDHHMMSS'
                    dt = datetime.datetime.strptime(f'{year_from_file}{day_of_year_from_file}{hour_from_file}{minute_from_file}{second_from_file}', '%Y%j%H%M%S')

                    # Nuevo nombre de archivo según el formato deseado
                    new_file_name = f'OR_{product_name}-M4_G16_s{dt.strftime("%Y%m%d%H%M%S")}.nc'
                    new_file_path = f'{path_dest}/{new_file_name}'

                    # Guardar una copia con el nuevo nombre
                    shutil.copy(cropped_file_path, new_file_path)
                    print(f'Renamed and copied file as {new_file_path}')
                    
                    # Eliminar el archivo original (nombre sin modificar)
                    if os.path.exists(cropped_file_path):
                        os.remove(cropped_file_path)
                        print(f'Deleted original file: {cropped_file_path}')
                    
            except Exception as e:
                print(f"Error processing file {local_file_path}: {e}")
            return local_file_path

    return None


# Función para procesar y reducir la escala de datos GOES para los tiempos específicos proporcionados en una lista
def process_goes_specific_times(path_local, path_dest, product_name, missing_times):
    path_dest = os.path.abspath(path_dest)
    if not os.path.exists(path_dest):
        os.makedirs(path_dest)

    # Iterar sobre cada fecha y hora exacta proporcionada en la lista
    for yyyymmddhhmn in missing_times:
        # Extraer la fecha y hora en los formatos necesarios
        date_save = yyyymmddhhmn[:8]
        time_save = yyyymmddhhmn[8:]

        # Procesar y reducir la resolución del archivo
        try:
            print(f"Processing for {date_save} at {time_save}")
            download_and_process_ACM(date_save + time_save, path_local, path_dest, product_name)
        except Exception as e:
            # Manejar cualquier error que ocurra durante el proceso
            print(f"Error processing for {date_save} at {time_save}: {e}")

# Define las rutas
temp_dir = 'content/temp_files/'
path_dest = 'C:/Users/guavitaj/Documents/tesis/algoritmos/modelo_xray/prueba/'
product_name = 'ABI-L2-ACMF'

# Llamar a la función para procesar solo los tiempos en la lista
process_goes_specific_times(temp_dir, path_dest, product_name, not_in_list)


Processing for 20230804 at 1910
No files found for the date: 202308041910
Processing for 20230804 at 1920
No files found for the date: 202308041920
Processing for 20230804 at 1930
No files found for the date: 202308041930
Processing for 20230819 at 2350
Cropped and saved file as C:\Users\guavitaj\Documents\tesis\algoritmos\modelo_xray\prueba/OR_ABI-L2-ACMF-M6_G16_s20232312350202_e20232312359510_c20232320000316.nc
Renamed and copied file as C:\Users\guavitaj\Documents\tesis\algoritmos\modelo_xray\prueba/OR_ABI-L2-ACMF-M4_G16_s20230819235020.nc
Deleted original file: C:\Users\guavitaj\Documents\tesis\algoritmos\modelo_xray\prueba/OR_ABI-L2-ACMF-M6_G16_s20232312350202_e20232312359510_c20232320000316.nc
Processing for 20230820 at 0000
Cropped and saved file as C:\Users\guavitaj\Documents\tesis\algoritmos\modelo_xray\prueba/OR_ABI-L2-ACMF-M6_G16_s20232320000202_e20232320009510_c20232320010339.nc
Renamed and copied file as C:\Users\guavitaj\Documents\tesis\algoritmos\modelo_xray\prueba/OR_AB